# Assignment 2


You must submit your notebook by running `python3 -m autograder.run.submit Assignment2.ipynb` from your local repository.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. To do so, restart the kernel and run all cells (in the menubar, select Runtime→→Restart and run all).

#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also **show how you arrived** at your answer.

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWERS HERE", as well as your name below:

## Some Helper Functions

Implement an evaluation function that takes in some board position and player colour and returns a score. 

In [ ]:
import chess
import random
from math import inf
from IPython.display import display, clear_output

## Q1

Implement the minimax algorithm that chooses the best move for a given board position, arbitrary evaluation function, player colour, and depth.

Make sure to write your implementation to be able to use the provided `evaluation()` function when calculating the value of board states. Later on, you'll be writing your own, but for now, your minimax will be tested using a provided evaluation function on the autograder.

Note about evaluation functions: call the evaluation function in the following manner: `evaluation(board: chess.board, player: bool)`. Think about which player should be passed into the evaluation function; should it be the player who is currently acting (according to the local max/min node) or should it be in accord with the player at the root of the tree?

If you time out during some of the test cases for this problem, consider adding alpha-beta pruning to your implementation.

In [8]:
import chess

def get_minimax_move(b: chess.Board, evaluation, player: bool, ply: int):
    """
    This function chooses the best move for the given board position, evaluation function, player, and ply.
    
    Parameters:
    - board: the chess board
    - evaluation: a function that returns a score given a chess board and a player
    - player: the colour of the active player (True -> white, False -> black)
    - ply: the number of move pairs (1 ply = both max AND min) that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """

    max_depth = 2 * ply     # total player moves, one for each

    def helper(board: chess.Board, depth: int, alpha: float, beta: float):
        if depth == 0 or board.is_game_over():
            return evaluation(board, player)
        
        maximizing = (board.turn == player)

        if maximizing:
            value = float('-inf')
            for move in board.legal_moves:
                board.push(move)
                value = max(value, helper(board, depth - 1, alpha, beta))
                board.pop()

                alpha = max(alpha, value)
                if alpha >= beta:
                    break

            return value
        else:
            value = float('inf')
            for move in board.legal_moves:
                board.push(move)
                value = min(value, helper(board, depth - 1, alpha, beta))
                board.pop()

                beta = min(beta,value)
                if beta <= alpha:
                    break

            return value
    
    best_move = None
    alpha = float('-inf')
    beta = float('inf')

    if b.turn == player:
        best_value = float('-inf')
    else:
        best_value = float('inf')

    for move in b.legal_moves:
        b.push(move)
        value = helper(b, max_depth - 1, alpha, beta)
        b.pop()

        if value > best_value:
            best_value = value
            best_move = move

        alpha = max(alpha, best_value)

    return best_move

def eval_simple(board, player):
    if board.is_checkmate():
        return float('inf') if board.turn != player else float('-inf')
    return 0

board = chess.Board("6k1/5ppp/8/8/8/8/5PPP/6KQ w - - 0 1")

move = get_minimax_move(board, eval_simple, chess.WHITE, ply=1)
print(move, board.san(move))


g1f1 Kf1


## Q2

Implement the expectimax algorithm that chooses the best move for a given board position, arbitrary evaluation function, player colour, and depth.

Make sure to write your implementation to be able to use the provided `evaluation()` function when calculating the value of board states. Later on, you'll be writing your own, but for now, your expectimax will be tested using a provided function on the autograder.

You should handle the evaluation function here in the same way as you did in your minimax implementation: `evaluation(board: chess.board, player: bool)`

In [ ]:
def get_expectimax_move(b: chess.Board, evaluation, player: bool, ply: int):   
    """
    This function chooses the best move for the given board position, evaluation function, player, and ply.
    
    Parameters:
    - board: the chess board
    - evaluation: a function that returns a score given a chess board and a player
    - player: the colour of the active player (True -> white, False -> black)
    - ply: the number of move pairs (1 ply = both max AND min) that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """
    max_depth = 2 * ply  # total half-moves

    def helper(board: chess.Board, depth: int):
        if depth == 0 or board.is_game_over():
            return evaluation(board, player)

        maximizing = (board.turn == player)

        legal_moves = list(board.legal_moves)  # important: copy generator

        if maximizing:
            value = float('-inf')
            for move in legal_moves:
                board.push(move)
                value = max(value, helper(board, depth - 1))
                board.pop()
            return value
        else:
            # chance node: average over all moves
            if not legal_moves:
                return evaluation(board, player)
            total = 0
            for move in legal_moves:
                board.push(move)
                total += helper(board, depth - 1)
                board.pop()
            return total / len(legal_moves)

    best_move = None
    best_value = float('-inf')

    for move in list(b.legal_moves):
        b.push(move)
        value = helper(b, max_depth - 1)
        b.pop()

        if value > best_value:
            best_value = value
            best_move = move

    return best_move

## Part 2

You will need to complete the implementation of your minimax function before continuing, as the puzzles that follow depend on its correctness.

In the previous questions, you wrote your implementation of minimax and tested it using a hidden evaluation function we wrote for you. Now, you are tasked with developing an original evaluation function `get_position_score` that will be used in conjunction with your minimax function. 

There are several chess puzzles below (drawn from `lichess.org`). Iterate on your evaluation function so that it can solve each puzzle as you come to it. During grading for each puzzle, you must solve the visible case as well as one hidden case to receive full credit. A general solution should be able to solve both.

In [ ]:
def get_position_score(board: chess.Board, player: bool):   
    """
    This function returns a score for a board position, from the given player's point of view.
    A higher score should be more favourable than a low score.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    
    Returns:
    A numerical value representing the score of the board position from player's point of view.
    """
    def _piece_value(piece_type):
        return {
            chess.PAWN: 100,
            chess.KNIGHT: 320,
            chess.BISHOP: 330,
            chess.ROOK: 500,
            chess.QUEEN: 900,
            chess.KING: 20000
        }.get(piece_type, 0)

    CENTER_SQUARES = {chess.D4, chess.D5, chess.E4, chess.E5}

    # --- Terminal positions ---
    if board.is_checkmate():
        return float('inf') if board.turn != player else float('-inf')
    if board.is_stalemate():
        return 0

    score = 0

    # --- Material and center control ---
    for piece_type in [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN]:
        # player pieces
        player_sqs = board.pieces(piece_type, player)
        score += sum(_piece_value(piece_type) + (20 if sq in CENTER_SQUARES else 0) for sq in player_sqs)
        # opponent pieces
        opp_sqs = board.pieces(piece_type, not player)
        score -= sum(_piece_value(piece_type) + (20 if sq in CENTER_SQUARES else 0) for sq in opp_sqs)

    # --- King safety: attackers on own king ---
    king_sq = board.king(player)
    if king_sq:
        attackers = board.attackers(not player, king_sq)
        score -= 50 * len(attackers)

    # --- Mobility: number of legal moves ---
    turn_backup = board.turn
    board.turn = player
    score += 5 * sum(1 for _ in board.legal_moves)  # small bonus
    board.turn = not player
    score -= 5 * sum(1 for _ in board.legal_moves)
    board.turn = turn_backup

    # --- 1-ply tactical evaluation: captures and checks ---
    # Only iterate over own moves
    for move in board.legal_moves:
        if board.turn != player:
            continue
        captured_piece = board.piece_at(move.to_square)
        if captured_piece and captured_piece.color != player:
            score += _piece_value(captured_piece.piece_type) // 2  # big bonus for captures
        if board.gives_check(move):
            score += 100  # bonus for giving check

    return score

### Q3

Given the puzzle below, modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, it will return a move resulting in checkmate.

<img src="images/asgn2%20q3%20puzzle.png" alt="assignment 2 question 3 puzzle figure" width="400"/>

In [41]:
# Feel free to use this code to test your solution to the puzzle above.
puzzle1_board = chess.Board("1rr5/p1p2Rpp/2Qpk3/4n1q1/4P3/8/PPP3PP/R6K")
move = get_minimax_move(puzzle1_board, get_position_score, True, 1)
print(f"Move: {move}")

Move: c6d5


#### Setting up an Engine

To generate an engine score, first you will need to download the chess engine 'Stockfish' from https://stockfishchess.org/download/.

Once that's done, you may find this docs page useful in setting it up: https://python-chess.readthedocs.io/en/latest/engine.html

Here, the engine score is given in 'Centipawns.' Modify the code below to match your own system and to confirm you have the engine set up correctly.

Note: you may notice that the scores returned by the engine are not consistent. That is, the same position may return a slightly different score. However, because the difference is small, it should not be an issue as the general trends hold true and the puzzles here tend to have an unambiguous best answer. As a failsafe, if your implementation returns the move we're looking for, you'll get full credit regardless of whatever score the engine returns.

In [35]:
import chess.engine
engine = chess.engine.SimpleEngine.popen_uci(r"/home/username/code/cse240/Assignment2/stockfish/stockfish-windows-x86-64-avx2.exe") 

# A completely standard board. White starts at a slight advantage, having first move.
board1 = chess.Board()
info1 = engine.analyse(board1, chess.engine.Limit(time=0.1))
print(f"Score: {info1['score'].white()}")

# A standard board, but White is missing both rooks. Naturally, White should have the disadvantage now.
board2 = chess.Board("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/1NBQKBN1")
info2 = engine.analyse(board2, chess.engine.Limit(time=0.1))
print(f"Score: {info2['score'].white()}")

engine.quit()

Score: +47
Score: -691


#### Q4
Given the puzzle below (White to move), modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, will return a good continuation to the game position. 

You cannot use the engine itself in your evaluation function, but you may find it helpful as you figure out what could make for good features in your evaluation function. Also recommended are the chess position analysis tools available online, such as `https://lichess.org/analysis/standard` to help you get some intuition about what sort of actions may be desirable. What kind of factors would you imagine go into scoring a chess board position? Try to implement some of those in your `get_position_score` function. 

<img src="images/asgn2%20q4%20puzzle.png" alt="assignment 2 question 34 puzzle figure" width="400"/>

In [40]:
# As above, so below; feel free to use this code to test your solution to the puzzle above.
puzzle2_board = chess.Board("8/1r3k2/2r1ppp1/8/5PB1/4P3/4PK2/5R2")
engine = chess.engine.SimpleEngine.popen_uci(r"/home/username/code/cse240/Assignment2/stockfish/stockfish-windows-x86-64-avx2.exe") 
pre_move_info = engine.analyse(puzzle2_board, chess.engine.Limit(time=0.1))

move = get_minimax_move(puzzle2_board, get_position_score, True, 3)
print(f"Move: {move}")
puzzle2_board.push(move)

post_move_info = engine.analyse(puzzle2_board, chess.engine.Limit(time=0.1))
engine.quit()
print(f"Pre-Move Score: {pre_move_info['score'].white()}")
print(f"Post-Move Score: {post_move_info['score'].white()}")

Move: g4f3
Pre-Move Score: 0
Post-Move Score: +2


#### Q5
Given the puzzle below (White to move), modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, will return a good continuation to the game position. 

You cannot use the engine itself in your evaluation function, but you may find it helpful as you figure out what could make for good features in your evaluation function. Also recommended are the chess position analysis tools available online, such as `https://lichess.org/analysis/standard` to help you get some intuition about what sort of actions may be desirable. What kind of factors would you imagine go into scoring a chess board position? Try to implement some of those in your `get_position_score` function. 

<img src="images/asgn2%20q5%20puzzle.png" alt="assignment 2 question 5 puzzle figure" width="400"/>

In [ ]:
# As above, so below; feel free to use this code to test your solution to the puzzle above.
puzzle3_board = chess.Board("1k6/2b2p2/2p1p3/1pP2p2/1P1P1P2/8/2N1P3/1K6")
engine = chess.engine.SimpleEngine.popen_uci(r"stockfish-windows-x86-64-avx2.exe")
pre_move_info = engine.analyse(puzzle3_board, chess.engine.Limit(time=0.1))

move = get_minimax_move(puzzle3_board, get_position_score, True, 1)
print(f"Move: {move}")
puzzle3_board.push(move)

post_move_info = engine.analyse(puzzle3_board, chess.engine.Limit(time=0.1))
engine.quit()
print(f"Pre-Move Score: {pre_move_info['score'].white()}")
print(f"Post-Move Score: {post_move_info['score'].white()}")